In [24]:
from seeq import spy
import pandas as pd
import numpy as np
import sys

spy.jobs.schedule('every 15 minutes')

,Schedule,Scheduled,Next Run
0,every 15 minutes,Every 15 minutes,2025-11-13 18:00:00 CST


,Schedule,Scheduled,Next Run
0,every 15 minutes,Every 15 minutes,2025-11-13 18:00:00 CST


In [25]:
#TIME SETTING 
user_tz = 'Asia/Shanghai'
now = pd.Timestamp.now(tz=user_tz)
start_time = now.floor('15min') - pd.Timedelta('10min')
end_time = now.floor('15min')
grid ='5min'
#取15min之前到25min之前之间10min的数据，有3个点

#以下是各个资产监控区别的信息，path是原数据的AF路径，asset是AF路径下的element name, signal names是attribute name（只挑选有效的tag)
path = "CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages)"
asset_name = "Fusheng_P-700_Beijing CG-SMIC BJ C01.2"
#这个清单的获取，请按照以下步骤获得
#将所有可用的参数拉到seeq worksheet里的趋势图里，并在AI输入口，输入：
#“请拉取左侧detailed pane里所有参数，并按照"Air Filter Delta Press.(STG1)","Air Filter Delta Press.(STG2)","Ambient Humidity",
#格式输出给我。

signal_names = [
   "Air Filter Delta Press.(STG1)",
"Air Filter Delta Press.(STG2)",
"Ambient Humidity",
"Ambient Temp.",
"MAC current",
"MAC discharge flow",
"Motor Bearing Temp.(loading side)",
"Motor Bearing Temp.(unloading side)",
"Motor Stator Temperture 1(u)",
"Oil Filter Delta Press.",
"Oil Press.(before filiter)",
"Oil Temp.",
"STG1 Cooler Outlet Water Temp.",
"STG1 discharge temp.",
"STG1 inlet temp.",
"STG2 Cooler Outlet Water Temp.",
"STG2 discharge temp.",
"STG2 inlet pressure",
"STG2 inlet temp.",
"STG3 discharge pressure",
"STG3 discharge temp.",
"STG3 inlet temp.",
"STG3/After Cooler Outlet Gas Temp.",
"STG3/After Cooler Outlet Water Temp.",
"Vibration STG1X",
"Vibration STG2X",
"Vibration STG3X",
"Cooling water temp.",
"IGV Openning CMD",
"STG3 inlet pressure"
]

In [29]:
# pre-filter
MAC_Current = spy.search({'Name': 'MAC current',
                         'Path': path,
                         'Asset': asset_name
                        })  
MAC_Current = spy.pull(
    MAC_Current,
    start=now.floor('15min'),
    end=now.floor('15min'),
    grid=grid
)
# 只有当数据有效时才继续执行
if pd.api.types.is_number(MAC_Current) and MAC_Current >= 10:
    # 这里写你的主要业务逻辑
    # 程序会正常继续执行
    pass
else:
    sys.exit(0)  # 数据无效时安静退出

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0F0A96F9-449E-64A0-8114-2FDD3CA4BB3A,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,MAC current,00:00:00.03,1,1,32 B,Success


SystemExit: 0

/usr/local/.pyenv/versions/3.11.13/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [31]:
all_results = []
for name in signal_names:
    result = spy.search({'Name': name,
                         'Path': path,
                         'Asset': asset_name
                        })
    all_results.append(result)

search_results = pd.concat(all_results, ignore_index=True)




data = spy.pull(
    search_results,
    start=start_time.isoformat(),
    end=end_time.isoformat(),
    grid=grid
)
data.columns = [col.split('>>')[-1].strip() for col in data.columns]
filtered_columns = [col for col in data.columns if ('Design' not in col) and ('AF' not in col)]
data_filtered = data[filtered_columns]
data_filtered = data_filtered[sorted(data_filtered.columns)]
data_filtered.head()

,ID,Type,Path,Asset,Name,Time,Count,Pages,Data Processed,Result
0,0F0A96F9-44A5-F9D0-A695-2445B3390CEB,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Air Filter Delta Press.(STG1),00:00:00.04,3,1,48 B,Success
1,0F0A96F9-44A5-F9D0-ADB8-E275A44A1197,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Air Filter Delta Press.(STG1)/Air Filter Delta Press.(STG1)_AF,00:00:00.06,3,1,16 B,Success
2,0F0A96F9-44A3-62C0-B283-056FD4232BB6,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Air Filter Delta Press.(STG2),00:00:00.05,3,1,48 B,Success
3,0F0A96F9-44A5-F9D0-A200-12F9EB6B73CA,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Air Filter Delta Press.(STG2)/Air Filter Delta Press.(STG2)_AF,00:00:00.26,3,1,16 B,Success
4,0F0A96F9-87F7-E800-8929-2542853D4A52,LiteralScalar,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Design Ambient Humidity,00:00:00.03,1,0,0 B,Success
5,0F0A96F9-44A3-62C0-8E8A-AC217DA65AC6,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Ambient Humidity,00:00:00.06,3,1,64 B,Success
6,0F0A96F9-87F5-60F0-BBB6-2C043CC9BBA9,LiteralScalar,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Design Ambient Temp.,00:00:00.01,1,0,0 B,Success
7,0F0A96F9-44A3-62C0-9510-9684765DD55D,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,Ambient Temp.,00:00:00.07,3,1,48 B,Success
8,0F0A96F9-449E-64A0-8114-2FDD3CA4BB3A,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,MAC current,00:00:00.07,3,1,32 B,Success
9,0F0A96F9-449E-64A0-AA00-594FE43C6D31,StoredSignal,CSTM_SIO.PAC(CN) >> 0_PAC >> Air Compressor(3 stages),Fusheng_P-700_Beijing CG-SMIC BJ C01.2,MAC discharge flow,00:00:00.09,3,1,32 B,Success


,Air Filter Delta Press.(STG1),Air Filter Delta Press.(STG2),Ambient Humidity,Ambient Temp.,Cooling water temp.,IGV Openning CMD,MAC current,MAC discharge flow,Motor Bearing Temp.(loading side),Motor Bearing Temp.(unloading side),...,STG2 inlet temp.,STG3 discharge pressure,STG3 discharge temp.,STG3 inlet pressure,STG3 inlet temp.,STG3/After Cooler Outlet Gas Temp.,STG3/After Cooler Outlet Water Temp.,Vibration STG1X,Vibration STG2X,Vibration STG3X
2025-11-13 17:35:00+08:00,NaN,NaN,NaN,NaN,16.301001,0,0,0,27.557103,26.957451,...,NaN,0,NaN,0,21.144374,17.902873,NaN,0,0.1,0.1
2025-11-13 17:40:00+08:00,NaN,NaN,NaN,NaN,16.241706,0,0,0,27.566971,26.964526,...,NaN,0,NaN,0,21.136075,17.873727,NaN,0,0.1,0.1
2025-11-13 17:45:00+08:00,0.214315,0.009477,38.378594,14.265021,16.201000,0,0,0,27.576840,26.971602,...,20.200001,0,22.700001,0,21.127776,17.835534,16.376616,0,0.1,0.1


In [ ]:

# 检查是否有任意一列不是数字类型
if any(not pd.api.types.is_numeric_dtype(data_filtered[col]) for col in data_filtered.columns):
    print("存在非数字类型的列，程序退出。")
    sys.exit(0)

In [ ]:
data_filtered.to_csv('latest_snapshot.csv')

In [ ]:
model_data_statistic=pd.read_csv('model_data_statistic.csv')

In [ ]:
means = model_data_statistic.iloc[0, 1:]  # 跳过第一列 Unnamed: 0
stds = model_data_statistic.iloc[1, 1:]

In [ ]:
# 新建一个空的DataFrame用于存放标准化结果
data_normalized = pd.DataFrame(index=data_filtered.index, columns=data_filtered.columns)

for col in data_filtered.columns:
        data_normalized[col] = (data_filtered[col] - means[col]) / stds[col]

In [ ]:
#data_normalized.to_csv('data_normalized.csv')

In [ ]:
# 读取CSV为DataFrame
df = pd.read_csv('cluster_centers.csv', index_col=0)

# 转为NumPy array
cluster_centers = df.to_numpy()  # 或 arr = df.to_numpy()
print(cluster_centers)

In [ ]:
len(cluster_centers)

In [ ]:
len(data_normalized.columns)

In [ ]:
temp = pd.DataFrame(index=data_normalized.index)
for j in range(len(cluster_centers)):
    center_series = pd.Series(cluster_centers[j], index=data_normalized.columns)
    # 计算每一行到该中心的欧氏距离
    temp['Dist_Cluster' + str(j)] = ((data_normalized - center_series) ** 2).sum(axis=1) ** 0.5

In [ ]:
# Creat the column of "Min_Dist" in model_data (whole data)
data_add_Min_Dist=data_normalized
data_add_Min_Dist['Min_Dist_'+asset_name]=temp.min(axis=1)
data_add_Min_Dist.columns

In [ ]:
Min_dist=data_add_Min_Dist['Min_Dist_'+asset_name].to_frame(name='Min_Dist_'+asset_name)

In [ ]:
Min_dist

In [ ]:
Min_Dist_mean_std=pd.read_csv('Min_Dist_mean_std.csv')
mean=Min_Dist_mean_std.loc[0, 'value']
std=Min_Dist_mean_std.loc[1, 'value']

In [ ]:
# 假设 Min_dist 是带时间戳的 DataFrame，只有一列 'Min_Dist'
# mean, std 已经是 float 类型

# 计算 mean+3*std
mean_plus_3std = mean + 3 * std
mean_minus_3std = mean - 3 * std

asset_mean ='mean_'+asset_name
asset_mean_minus_3std ='mean-3std_'+asset_name
asset_mean_plus_3std ='mean+3std_'+asset_name
# 构造一个新的 DataFrame，索引为和 Min_dist 一致
Min_dist[asset_mean] = mean
Min_dist[asset_mean_minus_3std] = mean_minus_3std
Min_dist[asset_mean_plus_3std] = mean_plus_3std
Min_dist

In [ ]:
data_add_Nearest_Cluster = data_add_Min_Dist
data_add_Nearest_Cluster['Nearest_Cluster']=temp.idxmin(axis=1)
data_add_Nearest_Cluster['Nearest_Cluster']=[int(x.replace('Dist_Cluster','')) for x in data_add_Nearest_Cluster['Nearest_Cluster']]
data_add_Nearest_Cluster['Nearest_Cluster']

In [ ]:
# Select the columns
effect_col=len(data_filtered.columns)
effect_col

In [ ]:
# Calculate the data error distribution for each variable in their cluster
data_add_distance = data_add_Nearest_Cluster
for i in range(effect_col):
    data_add_distance["d_" + data_add_Nearest_Cluster.columns[i]]=-100

In [ ]:
 data_add_distance

In [ ]:
# Calculate the data error distribution for each variable in their cluster
for j in range(len(cluster_centers)):
    center_series = pd.Series(cluster_centers[j],index=data_filtered.columns)
    temp_test= data_add_distance.loc[:, data_add_distance.columns[:effect_col]][ data_add_distance['Nearest_Cluster']==j]-center_series
    for i in range(effect_col):
        new_col_name1 = "d_" + data_filtered.columns[i]
        data_add_distance.loc[data_add_distance['Nearest_Cluster']==j,new_col_name1]=temp_test.iloc[:,i]

In [ ]:
# Calculate the distribution of square of error for each variable
data_add_Square_d = data_add_distance
data_add_Square_d['Cal_Square_Dist']=0

new_cols = {}

for i in range(effect_col):
    col_name = data_add_Square_d.columns[i]
    new_col_name = "Square_d_" + col_name
    new_cols[new_col_name] = data_add_Square_d["d_" + col_name] ** 2

# 一次性合并新列
data_add_Square_d = pd.concat([data_add_Square_d, pd.DataFrame(new_cols)], axis=1)

# 计算 Cal_Square_Dist
square_cols = [col for col in data_add_Square_d.columns if col.startswith("Square_d_")]
data_add_Square_d['Cal_Square_Dist'] = data_add_Square_d[square_cols].sum(axis=1)

In [ ]:
Cal_Square_Dist_col_indices = [i for i, col in enumerate(data_add_Square_d.columns) if 'Cal_Square_Dist' in col]
print(Cal_Square_Dist_col_indices)

In [ ]:
col_start_index = data_add_Square_d.columns.get_loc('Cal_Square_Dist')

In [ ]:
col_end_index=len(data_add_Square_d.columns)

In [ ]:
Contri_d=data_add_Square_d[data_add_Square_d.columns[col_start_index:col_end_index]].copy()

In [ ]:
for i in range(1, len(Contri_d.columns)):
    new_col = "Contri_" + Contri_d.columns[i]
    # 用 .loc 明确赋值
    Contri_d.loc[:, new_col] = Contri_d.iloc[:, i] / Contri_d.iloc[:, 0] * 100

In [ ]:
len(Contri_d.columns)

In [ ]:
# 找到所有以 "Square_d" 开头的列名的索引
square_d_indices = [i for i, col in enumerate(Contri_d.columns) if col.startswith("Contri_Square_d")]

# 获取最后一个的序号
if square_d_indices:
    # square_d_indices 已经包含了所有符合条件的索引
    first_square_d_index = square_d_indices[0]
    last_square_d_index = square_d_indices[-1]

In [ ]:
selected_columns = Contri_d.iloc[:, first_square_d_index:last_square_d_index +1]

In [ ]:
selected_columns = selected_columns.add_suffix('_'+asset_name)

In [ ]:
Min_dist = pd.concat([Min_dist, selected_columns], axis=1)

In [ ]:
#Min_dist.to_csv('Min_dist.csv')

In [ ]:
# Push the data point to the specified Workbook and Worksheet in Seeq
push_results = spy.push(data=Min_dist, workbook=None)